In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display
import numpy as np

# Load the dataset
df = pd.read_csv("full_year.csv", parse_dates=["timestamp"])
df.set_index("timestamp", inplace=True)

# Define available regions
regions = ["Auvergne_Rhone_Alpes", "Nouvelle_Aquitaine", "Occitanie", "Provence_Alpes_Cote_dAzur"]

# Create interactive widgets
date_range = widgets.SelectionRangeSlider(
    options=[d.strftime("%Y-%m-%d %H:%M:%S") for d in df.index.unique()],
    index=(0, len(df.index.unique()) - 1),
    description='Date Range',
    layout={'width': '95%'}
)

region_selector = widgets.Dropdown(
    options=regions,
    value="Occitanie",
    description='Région'
)

# Display widgets
display(date_range, region_selector)

def plot_all(date_range, region):
    start, end = pd.to_datetime(date_range)
    dff = df.loc[start:end].copy()

    # Mix de production
    prod_cols = [col for col in df.columns if col.startswith("dispatch_") and col.endswith(region)]
    fig1 = px.area(dff, y=prod_cols, title=f"Mix de production - {region}")
    fig1.show()

    # Stockage
    soc_cols = [f"storage_soc_STEP_{region}", f"storage_soc_batteries_{region}"]
    charge_cols = [f"storage_charge_STEP_{region}", f"storage_charge_batteries_{region}"]
    discharge_cols = [f"storage_discharge_STEP_{region}", f"storage_discharge_batteries_{region}"]
    fig2 = go.Figure()
    for col in soc_cols:
        fig2.add_trace(go.Scatter(x=dff.index, y=dff[col], mode='lines', name=col))
    fig2.update_layout(title=f"État de charge - {region}")
    fig2.show()

    fig3 = go.Figure()
    for col in charge_cols + discharge_cols:
        fig3.add_trace(go.Scatter(x=dff.index, y=dff[col], mode='lines', name=col))
    fig3.update_layout(title=f"Charge et décharge - {region}")
    fig3.show()

    # DR
    fig4 = go.Figure()
    fig4.add_trace(go.Scatter(x=dff.index, y=dff[f"demand_response_{region}"], mode='lines', name="Potentiel DR"))
    fig4.add_trace(go.Scatter(x=dff.index, y=dff[f"dr_active_{region}"], mode='lines', name="DR activé"))
    fig4.update_layout(title=f"Demande effaçable - {region}")
    fig4.show()

    # Slacks
    fig5 = go.Figure()
    fig5.add_trace(go.Scatter(x=dff.index, y=dff[f"slack_pos_{region}"], mode='lines', name="Slack +"))
    fig5.add_trace(go.Scatter(x=dff.index, y=dff[f"slack_neg_{region}"], mode='lines', name="Slack -"))
    fig5.update_layout(title=f"Slack positif/négatif - {region}")
    fig5.show()

    # Prix nodal
    fig6 = px.line(dff, y=[f"nodal_price_{region}"], title=f"Prix nodal - {region}")
    fig6.show()

    # Flux interrégionaux (positifs, négatifs, nets)
    flow_cols = [col for col in df.columns if col.startswith("flow_out_") and region in col]
    fig7 = go.Figure()
    for col in flow_cols:
        fig7.add_trace(go.Scatter(x=dff.index, y=dff[col], mode='lines', name=col.replace("flow_out_", "")))
    fig7.update_layout(title=f"Flux sortants de la région {region} vers les autres")
    fig7.show()

    incoming_cols = [col for col in df.columns if col.startswith("flow_out_") and col.endswith(region)]
    fig8 = go.Figure()
    for col in incoming_cols:
        fig8.add_trace(go.Scatter(x=dff.index, y=dff[col], mode='lines', name=col.replace("flow_out_", "")))
    fig8.update_layout(title=f"Flux entrants vers la région {region}")
    fig8.show()

    dff['flow_net'] = dff[incoming_cols].sum(axis=1) - dff[flow_cols].sum(axis=1)
    fig9 = px.line(dff, y='flow_net', title=f"Flux net pour la région {region} (positif = import net)")
    fig9.show()

    # Sankey diagram des flux interrégionaux moyens
    flows = [col for col in df.columns if col.startswith("flow_out_")]
    mean_flows = dff[flows].mean().reset_index()
    mean_flows.columns = ['flow', 'value']
    mean_flows = mean_flows[mean_flows['value'] > 0]

    def parse_flow(s):
        parts = s.replace("flow_out_", "").split("_")
        source = "_".join(parts[:-1])
        target = parts[-1]
        return source, target

    mean_flows[['source', 'target']] = mean_flows['flow'].apply(lambda x: pd.Series(parse_flow(x)))
    all_regions = list(set(mean_flows['source']).union(set(mean_flows['target'])))
    region_to_index = {region: i for i, region in enumerate(all_regions)}

    sankey_data = dict(
        type='sankey',
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color='black', width=0.5),
            label=all_regions
        ),
        link=dict(
            source=[region_to_index[src] for src in mean_flows['source']],
            target=[region_to_index[tgt] for tgt in mean_flows['target']],
            value=mean_flows['value'].round(2)
        )
    )

    fig10 = go.Figure(data=[sankey_data])
    fig10.update_layout(title_text=f"Diagramme de Sankey des flux interrégionaux moyens ({start.date()} à {end.date()})", font_size=10)
    fig10.show()

# Lier les widgets à la fonction
widgets.interactive_output(plot_all, {"date_range": date_range, "region": region_selector})


SelectionRangeSlider(description='Date Range', index=(0, 17472), layout=Layout(width='95%'), options=('2022-01…

Dropdown(description='Région', index=2, options=('Auvergne_Rhone_Alpes', 'Nouvelle_Aquitaine', 'Occitanie', 'P…

Output()